# Test Your Algorithm

## Instructions
1. From the **Pulse Rate Algorithm** Notebook you can do one of the following:
   - Copy over all the **Code** section to the following Code block.
   - Download as a Python (`.py`) and copy the code to the following Code block.
2. In the bottom right, click the <span style="color:blue">Test Run</span> button. 

### Didn't Pass
If your code didn't pass the test, go back to the previous Concept or to your local setup and continue iterating on your algorithm and try to bring your training error down before testing again.

### Pass
If your code passes the test, complete the following! You **must** include a screenshot of your code and the Test being **Passed**. Here is what the starter filler code looks like when the test is run and should be similar. A passed test will include in the notebook a green outline plus a box with **Test passed:** and in the Results bar at the bottom the progress bar will be at 100% plus a checkmark with **All cells passed**.
![Example](example.png)

1. Take a screenshot of your code passing the test, make sure it is in the format `.png`. If not a `.png` image, you will have to edit the Markdown render the image after Step 3. Here is an example of what the `passed.png` would look like 
2. Upload the screenshot to the same folder or directory as this jupyter notebook.
3. Rename the screenshot to `passed.png` and it should show up below.
![Passed](passed_ml.png)
4. Download this jupyter notebook as a `.pdf` file. 
5. Continue to Part 2 of the Project. 

In [ ]:
import glob

import numpy as np
import scipy as sp
import scipy.io
import glob
import scipy.stats
import scipy.signal
import os

def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]

def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Train the model
    train_model()
    
#     print("Start evaluating...")
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)

def RunPulseRateAlgorithm(data_fl, ref_fl):
    """ Calculates mean absolute errors and confidences
    
    Args:
        data_fl: (string) Path to the signal file
        ref_fl: (string) Path to the reference signal file
        
    Returns:
        (np.array) Mean absolute errors
        (np.array) Confidences
    """
    # Load data using LoadTroikaDataFile
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
    
    # load the reference signal
    ground_truth = scipy.io.loadmat(ref_fl)['BPM0'].reshape(-1)
    
    errs = []
    confs = []
    
    start_indices, end_indices = get_start_end(len(accx), len(ground_truth))
    
    for i, start in enumerate(start_indices):
        end = end_indices[i]
        ref = ground_truth[i]
        
        # Bandpass filtering the signals
        w_ppg =  bandpass_filter(ppg[start:end])
        w_accx = bandpass_filter(accx[start:end])
        w_accy = bandpass_filter(accy[start:end])
        w_accz = bandpass_filter(accz[start:end])
        
        # Get features
        feature = Featurize(w_ppg, w_accx, w_accy, w_accz)
        
        # Get prediction
        pred = model.predict(np.reshape(feature, (1, -1)))[0]
        
        # calculate confidence
        n = len(w_ppg) * 4
        freqs = np.fft.rfftfreq(n, 1/fs)
        fft = np.abs(np.fft.rfft(w_ppg,n))
        fft[freqs <= 40/60.0] = 0.0
        fft[freqs >= 240/60.0] = 0.0
    
        est_fs = pred / 55.0
        fs_win = 30  / 60.0
        fs_win_e = (freqs >= est_fs - fs_win) & (freqs <= est_fs +fs_win)
        conf = np.sum(fft[fs_win_e])/np.sum(fft)
        
        errs.append(np.abs(pred - ref))
        confs.append(conf)
    
    errs = np.array(errs)
    confs = np.array(confs)
    # Return mean absolute error and confidence as a 2-tuple of numpy arrays.
    return errs, confs
fs = 125
window_shift = 2
window_length = 10

def get_start_end(sig_len, ref_len):
    """ Returns the start and end indices of a signal """
    n = ref_len if ref_len < sig_len else sig_len
    start = (np.cumsum(np.ones(n) * fs * window_shift) - fs * window_shift).astype(int) # windows shift of 2
    return (start, start + window_length * fs) # windows length of 10

def bandpass_filter(signal, band_pass = (40/60, 240/60), fs = fs):
    """ Performs a bandpass filter on the signal. """
    
    b,a = scipy.signal.butter(3, band_pass, fs=fs, btype= 'bandpass')
    
    # Perform forward and backward digital butterworth filter
    return scipy.signal.filtfilt(b,a,signal)

def Featurize(ppg, accx, accy, accz):
    """ Create features """

    n = len(ppg) * 4
    freqs = np.fft.rfftfreq(n, 1/fs)
    fft = np.abs(np.fft.rfft(ppg,n))
    fft[freqs <= 40/60.0] = 0.0
    fft[freqs >= 240/60.0] = 0.0
    
    acc_mag = np.sqrt(accx**2 + accy**2 + accz**2)
    acc_fft = np.abs(np.fft.rfft(acc_mag, n))
    acc_fft[freqs <= 40/60.0] = 0.0
    acc_fft[freqs >= 240/60.0] = 0.0
    
    ppg_feature = freqs[np.argmax(fft)]
    acc_feature = freqs[np.argmax(acc_fft)]
    
    return np.array([ppg_feature, acc_feature])
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold

model = None

def train_model():
    global model
    
    if model: return
    
#     print("Starting...")
    data_fls, ref_fls = LoadTroikaDataset()
    
    targets, features, subjects = [], [], []
    
    for data_fl, ref_fl in (zip(data_fls, ref_fls)):
        
        # Load data using LoadTroikaDataFile
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)

        # load the reference signal
        ground_truth = scipy.io.loadmat(ref_fl)['BPM0'].reshape(-1)

        start_indices, end_indices = get_start_end(len(accx), len(ground_truth))
        
        subject_name = os.path.basename(data_fl).split('.')[0]  
        for i, start in enumerate(start_indices):
            end = end_indices[i]
            
            # Bandpass filtering the signals
            w_ppg =  bandpass_filter(ppg[start:end])
            w_accx = bandpass_filter(accx[start:end])
            w_accy = bandpass_filter(accy[start:end])
            w_accz = bandpass_filter(accz[start:end])

            # Get features
            feature = Featurize(w_ppg, w_accx, w_accy, w_accz)

            targets.append(ground_truth[i])
            features.append(feature)
            subjects.append(subject_name)
            
    targets = np.array(targets)
    features = np.array(features)
    
    # Start training
    regression = RandomForestRegressor(n_estimators=200,max_depth=15)
    lf = KFold(n_splits=5)
    splits = lf.split(features,targets,subjects)

    for i, (train_idx, test_idx) in enumerate(splits):
        X_train, y_train = features[train_idx], targets[train_idx]
        X_test, y_test = features[test_idx], targets[test_idx]
        regression.fit(X_train, y_train)
    
    model = regression
Evaluate()